# DuckDB


In [72]:
# NBVAL_IGNORE_OUTPUT
from sqlalchemy import sql
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, Sequence, DateTime
from datetime import datetime
from sqlalchemy import create_engine
import os

# To use DuckDB with siuba, just follow these steps! 
# 1. pip install duckdb
# 2. pip install duckdb_engine (install the DuckDB SQLAlchemy driver)

# For more details on DuckDB, visit https://www.duckdb.org

# This creates an in-memory duckdb, but a file-based one can also be created by replacing :memory: with a path to the file
engine = create_engine("duckdb:///:memory:")
# engine = create_engine('duckdb:////path/to/duck.db')

metadata = MetaData()
# The sequence is needed because SQLAlchemy defaults to using the legacy PostgreSQL data type of Serial, which DuckDB does not support
# See the readme of duckdb_engine for details: https://github.com/Mause/duckdb_engine
user_id_seq = Sequence('user_id_seq')
users = Table('users', metadata,
    Column('id', Integer, user_id_seq, server_default=user_id_seq.next_value(), primary_key=True),
    Column('name', String),
    Column('fullname', String),
)

# The sequence is needed because SQLAlchemy defaults to using the legacy PostgreSQL data type of Serial, which DuckDB does not support
# See the readme of duckdb_engine for details: https://github.com/Mause/duckdb_engine

# A DateTime field was also added in order to test date logic
address_id_seq = Sequence('address_id_seq')
addresses = Table('addresses', metadata,
  Column('id', Integer, address_id_seq,server_default=address_id_seq.next_value(), primary_key=True),
  Column('user_id', None, ForeignKey('users.id')),
  Column('email_address', String, nullable=False),
  Column('update_dt', DateTime)
 )

metadata.drop_all(engine)
metadata.create_all(engine)

conn = engine.connect()

ins = users.insert().values(name='jack', fullname='Jack Jones')
result = conn.execute(ins)


ins = users.insert()
conn.execute(ins, id=2, name='wendy', fullname='Wendy Williams')


conn.execute(addresses.insert(), [
   {'user_id': 1, 'email_address' : 'jack@yahoo.com', 'update_dt':datetime.now()},
   {'user_id': 1, 'email_address' : 'jack@msn.com', 'update_dt':datetime.now()},
   {'user_id': 2, 'email_address' : 'www@www.org', 'update_dt':datetime.now()},
   {'user_id': 2, 'email_address' : 'wendy@aol.com', 'update_dt':datetime.now()},
])


## Big Example

In [74]:
#If on Windows, be sure to pip install pyreadline
from siuba import *
from siuba.sql.verbs import LazyTbl, collect, show_query
from siuba.sql.dply.vector import dense_rank
import siuba.meta_hook.sqlalchemy.sql.functions as F

from sqlalchemy import sql

tbl_addresses = LazyTbl(conn, addresses)
tbl_users = LazyTbl(conn, users)

#tbl_addresses >> mutate(_, num = dense_rank(_.id)) >> show_query(_)
q = (tbl_addresses
  >> group_by("user_id")
  >> mutate(num = dense_rank(_.id))
  >> filter(
       _.id > _.id.min(),
       _.email_address.str.startswith("jack")
     )
  >> ungroup()
  >> show_query(simplify = True)
  >> collect()
  )

q

SELECT anon_1.id, anon_1.user_id, anon_1.email_address, anon_1.update_dt, anon_1.num 
FROM (SELECT id, user_id, email_address, update_dt, num, min(anon_2.id) OVER (PARTITION BY anon_2.user_id) AS win1, anon_2.id > min(anon_2.id) OVER (PARTITION BY anon_2.user_id) AS win2 
FROM (SELECT id, user_id, email_address, update_dt, dense_rank() OVER (PARTITION BY addresses.user_id ORDER BY addresses.id) AS num 
FROM addresses) AS anon_2) AS anon_1 
WHERE anon_1.win2 AND (anon_1.email_address LIKE 'jack' || '%')


C:\ProgramData\Anaconda3\envs\new_duckdb\lib\site-packages\siuba\sql\translate.py:24: SiubaSqlRuntimeWarning: 
dense_rank sql translation defaults na_option to None. To return identical result as pandas, use na_option = 'keep'.

This warning only displays once per function
  warnings.warn(


,id,user_id,email_address,update_dt,num
0,2,1,jack@msn.com,2022-01-19 08:12:01.020680,2


## Mutate

In [75]:
q = (tbl_addresses
  >> mutate(rank = dense_rank(_.id) + 1)
  >> show_query()
  )
q

SELECT addresses.id, addresses.user_id, addresses.email_address, addresses.update_dt, dense_rank() OVER (ORDER BY addresses.id) + 1 AS rank 
FROM addresses


C:\ProgramData\Anaconda3\envs\new_duckdb\lib\site-packages\siuba\sql\translate.py:24: SiubaSqlRuntimeWarning: 
dense_rank sql translation defaults na_option to None. To return identical result as pandas, use na_option = 'keep'.

This warning only displays once per function
  warnings.warn(


,id,user_id,email_address,update_dt,rank
0,1,1,jack@yahoo.com,2022-01-19 08:12:01.020680,2
1,2,1,jack@msn.com,2022-01-19 08:12:01.020680,3
2,3,2,www@www.org,2022-01-19 08:12:01.020680,4
3,4,2,wendy@aol.com,2022-01-19 08:12:01.020680,5


In [76]:
q = (tbl_addresses
  >> group_by("user_id")
  >> mutate(rank = _.id > dense_rank(_.id) + 1)
  >> show_query()
  )
q

SELECT addresses.id, addresses.user_id, addresses.email_address, addresses.update_dt, addresses.id > dense_rank() OVER (PARTITION BY addresses.user_id, addresses.user_id, addresses.user_id ORDER BY addresses.id) + 1 AS rank 
FROM addresses


C:\ProgramData\Anaconda3\envs\new_duckdb\lib\site-packages\siuba\sql\translate.py:24: SiubaSqlRuntimeWarning: 
dense_rank sql translation defaults na_option to None. To return identical result as pandas, use na_option = 'keep'.

This warning only displays once per function
  warnings.warn(


,id,user_id,email_address,update_dt,rank
0,3,2,www@www.org,2022-01-19 08:12:01.020680,True
1,4,2,wendy@aol.com,2022-01-19 08:12:01.020680,True
2,1,1,jack@yahoo.com,2022-01-19 08:12:01.020680,False
3,2,1,jack@msn.com,2022-01-19 08:12:01.020680,False


In [77]:
# rename and first mutate in same query,
# second mutate is outer query (since uses to prev col)
# Need to convert the boolean variable to an integer in order for DuckDB to be able to add 1 to it
q = (tbl_addresses
  >> select(_.email == _.email_address)
  >> mutate(is_mikey = _.email.str.startswith("mikey"), mikey2 = _.is_mikey.astype(int) + 1)
  >> show_query()
  )
q

SELECT anon_1.email, anon_1.is_mikey, CAST(anon_1.is_mikey AS INTEGER) + 1 AS mikey2 
FROM (SELECT anon_2.email AS email, (anon_2.email LIKE 'mikey' || '%') AS is_mikey 
FROM (SELECT addresses.email_address AS email 
FROM addresses) AS anon_2) AS anon_1


,email,is_mikey,mikey2
0,jack@yahoo.com,False,1
1,jack@msn.com,False,1
2,www@www.org,False,1
3,wendy@aol.com,False,1


## Filter

In [78]:
q = (tbl_addresses
  >> filter(_.id > 1)
  >> show_query()
  )
q

SELECT anon_1.id, anon_1.user_id, anon_1.email_address, anon_1.update_dt 
FROM (SELECT addresses.id AS id, addresses.user_id AS user_id, addresses.email_address AS email_address, addresses.update_dt AS update_dt 
FROM addresses) AS anon_1 
WHERE anon_1.id > 1


,id,user_id,email_address,update_dt
0,2,1,jack@msn.com,2022-01-19 08:12:01.020680
1,3,2,www@www.org,2022-01-19 08:12:01.020680
2,4,2,wendy@aol.com,2022-01-19 08:12:01.020680


In [79]:
q = (tbl_addresses
  >> group_by("user_id")
  >> filter(_.id > 1)
  >> show_query()
  )
q

SELECT anon_1.id, anon_1.user_id, anon_1.email_address, anon_1.update_dt 
FROM (SELECT addresses.id AS id, addresses.user_id AS user_id, addresses.email_address AS email_address, addresses.update_dt AS update_dt 
FROM addresses) AS anon_1 
WHERE anon_1.id > 1


,id,user_id,email_address,update_dt
0,2,1,jack@msn.com,2022-01-19 08:12:01.020680
1,3,2,www@www.org,2022-01-19 08:12:01.020680
2,4,2,wendy@aol.com,2022-01-19 08:12:01.020680


In [80]:
q = (tbl_addresses
  >> group_by("user_id")
  >> filter(dense_rank(_.id) > 1)
  >> show_query()
  >> collect()
  )

q

SELECT anon_1.id, anon_1.user_id, anon_1.email_address, anon_1.update_dt 
FROM (SELECT anon_2.id AS id, anon_2.user_id AS user_id, anon_2.email_address AS email_address, anon_2.update_dt AS update_dt, dense_rank() OVER (PARTITION BY anon_2.user_id, anon_2.user_id ORDER BY anon_2.id) AS win1, dense_rank() OVER (PARTITION BY anon_2.user_id, anon_2.user_id ORDER BY anon_2.id) > 1 AS win2 
FROM (SELECT addresses.id AS id, addresses.user_id AS user_id, addresses.email_address AS email_address, addresses.update_dt AS update_dt 
FROM addresses) AS anon_2) AS anon_1 
WHERE anon_1.win2


C:\ProgramData\Anaconda3\envs\new_duckdb\lib\site-packages\siuba\sql\translate.py:24: SiubaSqlRuntimeWarning: 
dense_rank sql translation defaults na_option to None. To return identical result as pandas, use na_option = 'keep'.

This warning only displays once per function
  warnings.warn(


,id,user_id,email_address,update_dt
0,4,2,wendy@aol.com,2022-01-19 08:12:01.020680
1,2,1,jack@msn.com,2022-01-19 08:12:01.020680


## Summarize

In [81]:
q = (tbl_addresses
  >> group_by("user_id")
  >> summarize(avg_id = _.id.mean())
  >> show_query()
  >> collect()
  )

q

SELECT addresses.user_id, avg(addresses.id) AS avg_id 
FROM addresses GROUP BY addresses.user_id


,user_id,avg_id
0,1,1.5
1,2,3.5


In [82]:
q = (tbl_addresses >> mutate(_, id2 = _.id + 1) >> summarize(_, m_id = _.id2.mean())) >> show_query()
q

SELECT avg(anon_1.id2) AS m_id 
FROM (SELECT addresses.id AS id, addresses.user_id AS user_id, addresses.email_address AS email_address, addresses.update_dt AS update_dt, addresses.id + 1 AS id2 
FROM addresses) AS anon_1


,m_id
0,3.5


## Count

In [83]:
q = (tbl_addresses
  >> group_by("user_id")
  >> count(_.id)
  >> show_query()
  >> collect()
)

q

SELECT anon_1.user_id, anon_1.id, count(*) AS n 
FROM (SELECT addresses.id AS id, addresses.user_id AS user_id, addresses.email_address AS email_address, addresses.update_dt AS update_dt 
FROM addresses) AS anon_1 GROUP BY anon_1.user_id, anon_1.id ORDER BY n DESC


,user_id,id,n
0,1,1,1
1,1,2,1
2,2,3,1
3,2,4,1


## Joins

In [84]:
# TODO: not executable like this, how to get first SELECT out of parens?
# E.g. can use users.join, etc..
q = (tbl_addresses
  >> left_join(_, tbl_users, {"user_id": "id"})
  >> show_query()
  >> collect()
  )

q

SELECT anon_1.id, anon_1.user_id, anon_1.email_address, anon_1.update_dt, anon_2.fullname, anon_2.name 
FROM (SELECT addresses.id AS id, addresses.user_id AS user_id, addresses.email_address AS email_address, addresses.update_dt AS update_dt 
FROM addresses) AS anon_1 LEFT OUTER JOIN (SELECT users.id AS id, users.name AS name, users.fullname AS fullname 
FROM users) AS anon_2 ON anon_1.user_id = anon_2.id


,id,user_id,email_address,update_dt,fullname,name
0,1,1,jack@yahoo.com,2022-01-19 08:12:01.020680,Jack Jones,jack
1,2,1,jack@msn.com,2022-01-19 08:12:01.020680,Jack Jones,jack
2,3,2,www@www.org,2022-01-19 08:12:01.020680,Wendy Williams,wendy
3,4,2,wendy@aol.com,2022-01-19 08:12:01.020680,Wendy Williams,wendy


## case_when

In [85]:
## TODO: fix sql case_when statements
q = (tbl_addresses
  >> mutate(
       label = case_when(_, {
         _.id > 20: 0,
         _.id > 1: 1,
         True: _.id
       })
     )
   >> show_query()
   >> collect()
  )

q

SELECT addresses.id, addresses.user_id, addresses.email_address, addresses.update_dt, CASE WHEN (addresses.id > 20) THEN 0 WHEN (addresses.id > 1) THEN 1 ELSE addresses.id END AS label 
FROM addresses


,id,user_id,email_address,update_dt,label
0,1,1,jack@yahoo.com,2022-01-19 08:12:01.020680,1
1,2,1,jack@msn.com,2022-01-19 08:12:01.020680,1
2,3,2,www@www.org,2022-01-19 08:12:01.020680,1
3,4,2,wendy@aol.com,2022-01-19 08:12:01.020680,1


In [86]:
## TODO: fix sql case_when statements
# works, but better to just use filter normally...
q = (tbl_addresses
  >> filter(
       case_when(_, {
         _.id > 20: True,
         _.id > 1: False,
         True: True
       })
     )
  >> show_query()
  >> collect()
  )

q

SELECT anon_1.id, anon_1.user_id, anon_1.email_address, anon_1.update_dt 
FROM (SELECT addresses.id AS id, addresses.user_id AS user_id, addresses.email_address AS email_address, addresses.update_dt AS update_dt 
FROM addresses) AS anon_1 
WHERE CASE WHEN (anon_1.id > 20) THEN true WHEN (anon_1.id > 1) THEN false ELSE true END


,id,user_id,email_address,update_dt
0,1,1,jack@yahoo.com,2022-01-19 08:12:01.020680


In [87]:
q = (tbl_addresses
  >> group_by("user_id")
  >> mutate(
       label = case_when(_, {
         _.id > _.id.mean(): 0,
         _.id > 20: 1,
         True: _.id
       })
     )
  >> show_query()
  >> collect()
  )

q

SELECT addresses.id, addresses.user_id, addresses.email_address, addresses.update_dt, CASE WHEN (addresses.id > avg(addresses.id) OVER (PARTITION BY addresses.user_id)) THEN 0 WHEN (addresses.id > 20) THEN 1 ELSE addresses.id END AS label 
FROM addresses


,id,user_id,email_address,update_dt,label
0,1,1,jack@yahoo.com,2022-01-19 08:12:01.020680,1
1,2,1,jack@msn.com,2022-01-19 08:12:01.020680,0
2,3,2,www@www.org,2022-01-19 08:12:01.020680,3
3,4,2,wendy@aol.com,2022-01-19 08:12:01.020680,0


In [88]:
# NBVAL_IGNORE_OUTPUT
case_when(_, {_.id > 1: "yeah", True: "no"})

█─'__call__'
├─<function case_when at 0x0000024A3A04AE50>
├─_
└─█─'<lazy>'
  └─█─'__call__'
    ├─<class 'dict'>
    └─{_.id > 1: 'yeah', True: 'no'}

## if_else

In [89]:
q = (tbl_addresses
  >> mutate(big_id = if_else(dense_rank(_.id) > 1, "yes", "no"))
  >> show_query()
  )
q

SELECT addresses.id, addresses.user_id, addresses.email_address, addresses.update_dt, CASE WHEN (dense_rank() OVER (ORDER BY addresses.id) > 1) THEN 'yes' ELSE 'no' END AS big_id 
FROM addresses


C:\ProgramData\Anaconda3\envs\new_duckdb\lib\site-packages\siuba\sql\translate.py:24: SiubaSqlRuntimeWarning: 
dense_rank sql translation defaults na_option to None. To return identical result as pandas, use na_option = 'keep'.

This warning only displays once per function
  warnings.warn(


,id,user_id,email_address,update_dt,big_id
0,1,1,jack@yahoo.com,2022-01-19 08:12:01.020680,no
1,2,1,jack@msn.com,2022-01-19 08:12:01.020680,yes
2,3,2,www@www.org,2022-01-19 08:12:01.020680,yes
3,4,2,wendy@aol.com,2022-01-19 08:12:01.020680,yes


## Head

In [90]:
(tbl_addresses
  >> head(3)
  >> show_query()
  >> collect()
  )

SELECT addresses.id, addresses.user_id, addresses.email_address, addresses.update_dt 
FROM addresses 
 LIMIT 3


,id,user_id,email_address,update_dt
0,1,1,jack@yahoo.com,2022-01-19 08:12:01.020680
1,2,1,jack@msn.com,2022-01-19 08:12:01.020680
2,3,2,www@www.org,2022-01-19 08:12:01.020680


## Rename

In [91]:
(tbl_addresses
  >> rename(id2 = "id")
  >> show_query()
  >> collect()
  )

SELECT addresses.id AS id2, addresses.user_id, addresses.email_address, addresses.update_dt 
FROM addresses


,id2,user_id,email_address,update_dt
0,1,1,jack@yahoo.com,2022-01-19 08:12:01.020680
1,2,1,jack@msn.com,2022-01-19 08:12:01.020680
2,3,2,www@www.org,2022-01-19 08:12:01.020680
3,4,2,wendy@aol.com,2022-01-19 08:12:01.020680


## Distinct

In [92]:
(tbl_addresses
  >> distinct(_.user_id, user_id2 = _.user_id + 1)
  >> show_query()
  >> collect()
  )

SELECT DISTINCT addresses.user_id, addresses.user_id + 1 AS user_id2 
FROM addresses


,user_id,user_id2
0,1,2
1,2,3


In [93]:
(tbl_addresses
  >> group_by("user_id")
  >> distinct(_.email_address, user_id2 = dense_rank(_.user_id))
  >> show_query()
  >> collect()
  )

SELECT DISTINCT addresses.email_address, dense_rank() OVER (PARTITION BY addresses.user_id ORDER BY addresses.user_id) AS user_id2 
FROM addresses


C:\ProgramData\Anaconda3\envs\new_duckdb\lib\site-packages\siuba\sql\translate.py:24: SiubaSqlRuntimeWarning: 
dense_rank sql translation defaults na_option to None. To return identical result as pandas, use na_option = 'keep'.

This warning only displays once per function
  warnings.warn(


,email_address,user_id2
0,jack@yahoo.com,1
1,jack@msn.com,1
2,www@www.org,1
3,wendy@aol.com,1


# Technical

## Translating symbolic function calls

In [94]:
from siuba.dply.vector import n

tbl_addresses \
  >> group_by("user_id") \
  >> summarize(n = n(_)) \
  >> collect()

,user_id,n
0,1,2
1,2,2


## Translating str methods

## Translating dt methods

In [95]:
import pandas as pd
# pd.read_sql("""select * from information_schema.tables""",conn)
# pd.read_sql("""select * from pragma_table_info('addresses')""",conn)
# pd.read_sql("""select * from addresses""",conn)
pd.read_sql("""SELECT addresses.id, addresses.user_id, addresses.email_address, EXTRACT(hour FROM current_timestamp) AS hour 
FROM addresses""",conn)

,id,user_id,email_address,hour
0,1,1,jack@yahoo.com,15
1,2,1,jack@msn.com,15
2,3,2,www@www.org,15
3,4,2,wendy@aol.com,15


In [96]:
q = tbl_addresses >> mutate(hour = _.update_dt.dt.hour) >> show_query()
q

SELECT addresses.id, addresses.user_id, addresses.email_address, addresses.update_dt, EXTRACT(hour FROM addresses.update_dt) AS hour 
FROM addresses


,id,user_id,email_address,update_dt,hour
0,1,1,jack@yahoo.com,2022-01-19 08:12:01.020680,8
1,2,1,jack@msn.com,2022-01-19 08:12:01.020680,8
2,3,2,www@www.org,2022-01-19 08:12:01.020680,8
3,4,2,wendy@aol.com,2022-01-19 08:12:01.020680,8


## SQL escapes

## Window functions

In [97]:
from siuba.dply.vector import desc
(tbl_addresses
  >> arrange(desc(_.id))
  >> mutate(cumsum = _.user_id.cumsum())
  >> arrange(_.cumsum)
  >> show_query()
  )

SELECT anon_1.id, anon_1.user_id, anon_1.email_address, anon_1.update_dt, sum(anon_1.user_id) OVER (ORDER BY anon_1.id DESC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cumsum 
FROM (SELECT addresses.id AS id, addresses.user_id AS user_id, addresses.email_address AS email_address, addresses.update_dt AS update_dt 
FROM addresses ORDER BY addresses.id DESC) AS anon_1 ORDER BY cumsum


,id,user_id,email_address,update_dt,cumsum
0,4,2,wendy@aol.com,2022-01-19 08:12:01.020680,2
1,3,2,www@www.org,2022-01-19 08:12:01.020680,4
2,2,1,jack@msn.com,2022-01-19 08:12:01.020680,5
3,1,1,jack@yahoo.com,2022-01-19 08:12:01.020680,6


# Misc

## postgres specific

In [98]:
#import pandas as pd
pd.read_sql("""SELECT addresses.id, addresses.user_id, addresses.email_address, round(CAST(addresses.id AS NUMERIC), 2) AS id2 
FROM addresses""",conn)

,id,user_id,email_address,id2
0,1,1,jack@yahoo.com,1.0
1,2,1,jack@msn.com,2.0
2,3,2,www@www.org,3.0
3,4,2,wendy@aol.com,4.0


In [99]:
(tbl_addresses
  >> mutate(id2 = _.id.round(2))
  >> show_query()
  >> collect()
)


SELECT addresses.id, addresses.user_id, addresses.email_address, addresses.update_dt, round(CAST(addresses.id AS NUMERIC), 2) AS id2 
FROM addresses


NotImplementedError: Not implemented Error: ROUND(DECIMAL, INTEGER) with non-constant precision is not supported

## autoload table w/ sqlalchemy

In [100]:
import sqlalchemy

metadata2 = MetaData()

#Since we are using an in memory DuckDB that is specific to the connection, need to autoload_with=conn not =engine
users2 = sqlalchemy.Table('users', metadata2, autoload = True, autoload_with = conn)
tbl_users2 = LazyTbl(conn, users2)

(tbl_users
  >> mutate(id2 = _.id + 1)
  >> show_query()
  >> collect()
  )

SELECT users.id, users.name, users.fullname, users.id + 1 AS id2 
FROM users


,id,name,fullname,id2
0,1,jack,Jack Jones,2
1,2,wendy,Wendy Williams,3


## auto table from string

In [101]:
import sqlalchemy

metadata3 = MetaData()

tbl_users3 = LazyTbl(conn, "users")

(tbl_users
  >> mutate(id2 = _.id + 1)
  >> show_query()
  >> collect()
  )

SELECT users.id, users.name, users.fullname, users.id + 1 AS id2 
FROM users


,id,name,fullname,id2
0,1,jack,Jack Jones,2
1,2,wendy,Wendy Williams,3


##  LazyTbl repr

In [102]:
tbl_users >> mutate(id2 = _.id + 1)

,id,name,fullname,id2
0,1,jack,Jack Jones,2
1,2,wendy,Wendy Williams,3
